In [1]:
import os
import re
import glob

import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import random_split
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import numpy as np
import segmentation_models_pytorch as smp

#from loss import *
from load import *
from run_length_encoding import *
import ttach as tta


In [2]:
#gpu setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Select the GPU 0,1 or both to use

In [3]:
#get gpu_0
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def get_model(model_str: str):
    if model_str == 'Unet':
        return smp.Unet
    elif model_str == 'FPN':
        return smp.FPN
    elif model_str == 'DeepLabV3Plus':
        return smp.DeepLabV3Plus
    elif model_str == 'UnetPlusPlus':
        return smp.UnetPlusPlus
    elif model_str == 'PAN':
        return smp.PAN
    elif model_str == 'MAnet':
        return smp.MAnet
    elif model_str == 'PSPNet':
        return smp.PSPNet
    
    
def get_optimizer(optimizer_str: str):
    if optimizer_str == 'SGD':
        optimizer = torch.optim.SGD
    elif optimizer_str == 'Adam':
        optimizer = torch.optim.Adam
    elif optimizer_str == 'AdamW':
        optimizer = torch.optim.AdamW
    else:
        optimizer = None

    return optimizer

def extract_numbers(string):
    numbers = re.findall(r'\d+', string)
    numbers_list = np.array([int(number) for number in numbers])
    #print(numbers_list)
    return np.max(numbers_list)



# 가장 최근에 저장된 모델의 weight를 가져옵니다.
def get_latest_checkpoint(checkpoint_dir):
    checkpoint_files = glob.glob(os.path.join(checkpoint_dir, '*'))
    if not checkpoint_files:
        raise FileNotFoundError(f"No checkpoint files found in {checkpoint_dir}")
        return 0, None

    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    last_epoch = extract_numbers(latest_checkpoint)
    
    return last_epoch, latest_checkpoint


def get_weight(path, is_dir=False): 
    if is_dir == True:
        '''
        path : model weight가 저장된 디렉토리
        '''
        last_epoch, last_ckpt_path = get_latest_checkpoint(path)
        if last_epoch == 0:
            return False
        else:
            last_ckpt = torch.load(last_ckpt_path)
            return (last_epoch, last_ckpt)
    else:
        print('load \'{}\''.format(path))
        return torch.load(path)

In [5]:
def get_transform_for_test():
    transform = A.Compose(
        [   
            A.MinMaxNormalize(),
            ToTensorV2()
        ]
    )
    return transform

# MODEL LOAD

1. model 관련 parameter확인
2. 저장된 모델 경로 수정
3. run 후 로드된 모델 확인
4. 저장 될 submit.csv 파일 경로 및 이름 수정

In [6]:
ARCHITECTURE = 'UnetPlusPlus'
ENCODER = 'resnet152' #timm-regnety_016
ENCODER_WEIGHT= 'imagenet' #imagenet
N_CLASSES = 1
ACTIVATION = None
SAVED_MODEL_PATH = '/root/jupyter/Dacon/deeplabv3p/model_save_unet_1/weight_epoch-0111.pth'

ckpt = torch.load(SAVED_MODEL_PATH)
model = get_model(ARCHITECTURE)
model = model(classes=N_CLASSES,
                    encoder_name=ENCODER,
                    encoder_weights=ENCODER_WEIGHT,
                    activation=ACTIVATION)

model = nn.DataParallel(model)
model.load_state_dict(ckpt, strict=False)

<All keys matched successfully>

In [7]:
model.to(DEVICE)

DataParallel(
  (module): UnetPlusPlus(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

## 돌리기 전에 csv 파일 이름 변경

submit_{model_epoch}.csv

In [8]:
transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles = [0, 180]),
        tta.Resize(sizes = [(448,448)], original_size=(224,224), interpolation= "nearest")
        
    ]
)

In [9]:
tta_model = tta.SegmentationTTAWrapper(model, transforms,merge_mode= "mean")

In [10]:
from skimage import io, morphology
transform_test = get_transform_for_test()
test_dataset = SatelliteDataset(csv_file='/root/jupyter/Dacon/deeplabv3p/test.csv', transform=transform_test, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

In [11]:
with torch.no_grad():
    tta_model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(DEVICE)
        
        # 1. model output
        outputs = tta_model(images)
        
        # 2. activation func and thresholding
        masks = torch.sigmoid(outputs).cpu().numpy()
        
        masks = (masks > .5).astype(np.uint8) 
       
        
        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)
            
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result

# change the file name.
# format : submit_{model_epoch}.csv
submit.to_csv(f'./submit/submit_{ARCHITECTURE}_{ENCODER}_{218}.csv', index=False)

 34% 204/607 [17:14<34:03,  5.07s/it]


KeyboardInterrupt: 

# HE submit

In [7]:
transform_test = get_transform_for_test()
test_dataset = SatelliteDataset_HE(csv_file='/root/jupyter/Dacon/deeplabv3p/test.csv', transform=transform_test, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

In [8]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(DEVICE)
        
        # 1. model output
        outputs = model(images)
        
        # 2. activation func and thresholding
        masks = torch.sigmoid(outputs).cpu().numpy()
        
        masks = (masks > .5).astype(np.uint8) 
       
        
        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)
            
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result

# change the file name.
# format : submit_{model_epoch}.csv
submit.to_csv(f'./submit/submit_{ARCHITECTURE}_{ENCODER}_{218}_HE.csv', index=False)

100% 60640/60640 [22:51<00:00, 44.22it/s]


In [12]:
submit.head()

,img_id,mask_rle
0,TEST_00000,19553 7 19777 9 19809 4 20001 10 20032 7 20224...
1,TEST_00001,-1
2,TEST_00002,-1
3,TEST_00003,18 26 73 9 243 25 297 8 465 28 522 5 689 29 91...
4,TEST_00004,16891 19 17114 23 17152 4 17337 44 17562 44 17...


# Opening after make Submit

In [20]:
opening_mask = []
for rle in submit.mask_rle:
    image = rle_decode(str(rle), (224,224))
    opening = morphology.binary_opening(image, morphology.square(3))
    opening_mask.append(opening)

result = []
for i in range(len(opening_mask)):
        mask_rle = rle_encode(opening_mask[i])
        if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
            result.append(-1)
        else:
            result.append(mask_rle)
            
submit = pd.read_csv('./train_28049_2.csv')
submit['mask_rle'] = result

# change the file name.
# format : submit_{model_epoch}.csv
submit.to_csv('./submit/submit_unet-resnet152_100epochs_opening.csv', index=False)

In [ ]:
df = pd.read_csv('./submit/')

In [ ]:
from skimage import io, morphology

opening = morphology.binary_opening(image, morphology.square(3))
